# Curso de Python do Zero ao DS
## Profº Meigarom
### Canal: Seja um Data Scientist


## Agenda

* 1.0 - O Problema do negócio
  * 1.1 - A Empresa: House Rocket
  * 1.2 - As perguntas do contratante (CEO)
* 2.0 - Planejamento da solução
  * 2.1 - Planejamento do produto final
  * 2.2 - Planejamento das ferramentas
  * 2.3 - Planejamento do processo
* 3.0 - Solução
* 4.0 - Entrega

# 1.0 - O Problema do negócio

## 1.1 - A Empresa: House Rocket
* O que faz? 
    * Plataforma de compras e vendas de imóveis.
* Qual é o problema? 
    * O CEO da empresa gostaria de maximizar os lucros encontrando bons negócios.
* Qual é a principal estratégia?
    * Fontes externas para encontrar bons negócios.

## 1.2 - As perguntas do contratante (CEO):

1. Qual a média do preço de compra dos imóveis por “Nível”? 
- Nível 0 -> Preço entre R$ 0 e R$ 321.950
- Nível 1 -> Preço entre R$ 321.950 e R$ 450.000
- Nível 2 -> Preço entre R$ 450.000 e R$ 645.000
- Nível 3 -> Acima de R$ 645.000

2. Qual a média do tamanho da sala de estar dos imóveis por “Size” ?
- Size 0 -> Tamanho entre 0 e 1427 sqft
- Size 1 -> Tamanho entre 1427 e 1910 sqft
- Size 2 -> Tamanho entre 1910 e 2550 sqft
- Size 3 -> Tamanho acima de 2550 sqft

3. Adicione as seguinte informações ao conjunto de dados original:
- Place ID: Identificação da localização 
- OSM Type: Open Street Map type
- Country: Nome do País
- Country Code: Código do País

4. Adicione os seguinte filtros no Mapa:
- Tamanho mínimo da área da sala de estar.
- Número mínimo de banheiros.
- Valor Máximo do Preço.
- Tamanho máximo da área do porão.
- Filtro das Condições do Imóvel.
- Filtro por Ano de Construção.

5. Adicione os seguinte filtros no Dashboard:
- Filtro por data disponível para compra.
- Filtro por ano de renovação.
- Filtro se possui vista para a água ou não.

# 2.0 - Planejamento da solução
## 2.1 - Planejamento do produto final
* Email+3 anexos:
  * Email: Pergunta | Resposta;
  * Anexo1: Arquivo .csv com as novas informações requisitadas;
  * Anexo2: Um mapa com os filtros requisitados;
  * Anexo3: Um dashboard com os filtros requisitados.

## 2.2 - Planejamento das ferramentas
* Python 3.9.2
* Jupyter Notebook
* VSCode

## 2.3 - Planejamento do processo

In [24]:
# Carregar bibliotecas e dataset
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import gridspec
from matplotlib import pyplot as plt
import ipywidgets as widgets
import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual

from geopy.geocoders import Nominatim
import time
import defs
from multiprocessing import Pool

In [25]:
#  Supress Scientific Notation
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', '{:.2f}'.format)

In [26]:
# Loading data into memory
df = pd.read_csv('datasets/kc_house_data.csv')

# Garantir que o formato date é um datetime
df['date'] = pd.to_datetime( df['date'], format='%Y-%m-%d' )

df.dtypes



id                        int64
date             datetime64[ns]
price                   float64
bedrooms                  int64
bathrooms               float64
sqft_living               int64
sqft_lot                  int64
floors                  float64
waterfront                int64
view                      int64
condition                 int64
grade                     int64
sqft_above                int64
sqft_basement             int64
yr_built                  int64
yr_renovated              int64
zipcode                   int64
lat                     float64
long                    float64
sqft_living15             int64
sqft_lot15                int64
dtype: object

1. Qual a média do preço de compra dos imóveis por “Nível”? 
- Nível 0 -> Preço entre R$ 0 e R$ 321.950
- Nível 1 -> Preço entre R$ 321.950 e R$ 450.000
- Nível 2 -> Preço entre R$ 450.000 e R$ 645.000
- Nível 3 -> Acima de R$ 645.000

>>  Criar coluna 'nível' com a classificação mediante o intervalo de prços da instrução acima;


In [27]:
df['nivel'] = 'NA'

In [28]:
%%timeit
df['nivel'] = df['price'].apply( lambda x: 
                'Nivel 0'   if x < 321950.00 else 
                'Nivel 1'   if (x>=321950.00) & (x<450000.00) else
                'Nivel 2'   if (x>=450000.00) & (x<645000.00) else 
                'Nivel 3' )
df[['price','nivel']]

7.96 ms ± 786 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
%%timeit

df.loc[df['price']<321950.00,'nivel'] = 'Nivel 0'
df.loc[(df['price']>=321950.00) & (df['price']<450000.00),'nivel'] = 'Nivel 1'
df.loc[(df['price']>=450000.00) & (df['price']<645000.00),'nivel'] = 'Nivel 2'
df.loc[df['price']>=645000.00,'nivel'] = 'Nivel 3'

df[['price','nivel']]

3.32 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


>> Agrupar por 'nivel' e calcular a média dos preços.


In [30]:
df[['price','nivel']].groupby('nivel').mean().reset_index()

,nivel,price
0,Nivel 0,251544.62
1,Nivel 1,383585.21
2,Nivel 2,536164.42
3,Nivel 3,985008.98


2. Qual a média do tamanho da sala de estar dos imóveis por “Size” ?
- Size 0 -> Tamanho entre 0 e 1427 sqft
- Size 1 -> Tamanho entre 1427 e 1910 sqft
- Size 2 -> Tamanho entre 1910 e 2550 sqft
- Size 3 -> Tamanho acima de 2550 sqft

>> Criar coluna 'size' classificando os imóveis de acordo com a tabela de tamanho;
>>
>> Agrupar por 'size' e calcular a média do tamanho da sala.

In [31]:
df['size'] = "NA"

In [32]:
%%timeit
df.loc[df['sqft_living']<1427, 'size'] = 'Size 0'
df.loc[(df['sqft_living']>=1427) & (df['sqft_living']<1910),'size'] = 'Size 1'
df.loc[(df['sqft_living']>=1910) & (df['sqft_living']<2550),'size'] = 'Size 2'
df.loc[df['sqft_living']>2550, 'size'] = 'Size 3'
df[['sqft_living','size']]

3.28 ms ± 279 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


3. Adicione as seguinte informações ao conjunto de dados original:
- Place ID: Identificação da localização 
- OSM Type: Open Street Map type
- Country: Nome do País
- Country Code: Código do País

In [33]:
df['query'] = df[['lat', 'long']].apply( lambda x: str( x['lat'] ) + ',' + str( x['long'] ), axis=1 )
df1 = df[['id', 'query']].head()
df1
p = Pool(2)
start = time.process_time()
df1[['place_id', 'osm_type', 'country', 'country_code']] = p.map( defs.get_longlat, df1.iterrows() )
print(time.process_time() - start)

0.016328795999999812


In [34]:
df1.head()

,id,query,place_id,osm_type,country,country_code
0,7129300520,"47.5112,-122.257",164281254,way,United States,us
1,6414100192,"47.721,-122.319",162919585,way,United States,us
2,5631500400,"47.7379,-122.233",74350607,node,United States,us
3,2487200875,"47.5208,-122.393",160391552,way,United States,us
4,1954400510,"47.6168,-122.045",309183460,way,United States,us


4. Adicione os seguinte filtros no Mapa:
- Tamanho mínimo da área da sala de estar.
- Número mínimo de banheiros.
- Valor Máximo do Preço.
- Tamanho máximo da área do porão.
- Filtro das Condições do Imóvel.
- Filtro por Ano de Construção.

In [35]:
df.dtypes

id                        int64
date             datetime64[ns]
price                   float64
bedrooms                  int64
bathrooms               float64
sqft_living               int64
sqft_lot                  int64
floors                  float64
waterfront                int64
view                      int64
condition                 int64
grade                     int64
sqft_above                int64
sqft_basement             int64
yr_built                  int64
yr_renovated              int64
zipcode                   int64
lat                     float64
long                    float64
sqft_living15             int64
sqft_lot15                int64
nivel                    object
size                     object
query                    object
dtype: object

In [36]:
# Prepare Data
df['is_waterfront'] = df['waterfront'].apply( lambda x: 'yes' if x == 1 else 'no' )


# define level of prices
df['level'] = df['price'].apply( lambda x: 0 if x <= 321950 else
                        1 if ( x > 321950 ) & ( x <= 450000) else
                        2 if ( x > 450000 ) & ( x <= 645000) else 3 )

df['level'] = df['level'].astype( int )


In [37]:
# Iteractive buttons
price_limit = widgets.IntSlider(
                value = 540000,
                min = 75000,
                max = 7700000,
                step = 1,
                description='Maximun Price',
                disable=False,
                style={'description_width': 'initial'} 
                )

waterfront_bar = widgets.Dropdown(
                options = df['is_waterfront'].unique().tolist(),
                value = 'yes',
                description = 'Water View',
                disable=False
                )

livingroom_limit = widgets.IntSlider(
                value = int( df['sqft_living'].mean() ),
                min = df['sqft_living'].min(),
                max = df['sqft_living'].max(),
                step = 1,
                description='Minimum Living Room Size',
                disable=False,
                style={'description_width': 'initial'}
                )

bathroom_limit = widgets.IntSlider(
                value = int( df['bathrooms'].mean() ),
                min = df['bathrooms'].min(),
                max = df['bathrooms'].max(),
                step = 1,
                description='Minimum Bathroom Number',
                disable=False,
                style={'description_width': 'initial'}
                )

basement_limit = widgets.IntSlider(
                value = int( df['sqft_basement'].mean() ),
                min = df['sqft_basement'].min(),
                max = df['sqft_basement'].max(),
                step = 1,
                description='Minimum Basement Size',
                disable=False,
                style={'description_width': 'initial'}
                )

condition_limit = widgets.IntSlider(
                value = int( df['condition'].mean() ),
                min = df['condition'].min(),
                max = df['condition'].max(),
                step = 1,
                description='Minimum condition',
                disable=False,
                style={'description_width': 'initial'}
                )

year_limit = widgets.IntSlider(
                value = int( df['yr_built'].mean() ),
                min = df['yr_built'].min(),
                max = df['yr_built'].max(),
                step = 1,
                description='Year Built',
                disable=False,
                style={'description_width': 'initial'}
                )

In [38]:
def update_map( df, waterfront, limit, livingroom_limit, bathroom_limit,basement_limit, condition_limit, year_limit ):
    houses = df[(df['price'] <= limit) &
                (df['is_waterfront'] == waterfront) &
                (df['sqft_living'] >= livingroom_limit) &
                (df['bathrooms'] >= bathroom_limit) &
                (df['sqft_basement'] >= basement_limit) &
                (df['condition'] >= condition_limit) &
                (df['yr_built'] >= year_limit )][['id', 'lat', 'long', 'price','level']]

    fig = px.scatter_mapbox(    houses,
                                lat="lat",
                                lon="long",
                                size="price",
                                color="level",
                                color_continuous_scale=px.colors.cyclical.IceFire,
                                size_max=15,
                                zoom=10)
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(height=600, margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

In [39]:
widgets.interactive(    update_map,
                        df=fixed( df ),
                        waterfront=waterfront_bar,
                        limit=price_limit,
                        livingroom_limit=livingroom_limit,
                        bathroom_limit=bathroom_limit,
                        basement_limit=basement_limit,
                        condition_limit=condition_limit,
                        year_limit=year_limit
                    )

interactive(children=(Dropdown(description='Water View', index=1, options=('no', 'yes'), value='yes'), IntSlid…

5. Adicione os seguinte filtros no Dashboard:
- Filtro por data disponível para compra.
- Filtro por ano de renovação.
- Filtro se possui vista para a água ou não.

In [40]:
# Change data format
df['year'] = pd.to_datetime( df['date'] ).dt.strftime( '%Y' )
df['date'] = pd.to_datetime( df['date'] ).dt.strftime( '%Y-%m-%d' )
df['year_week'] = pd.to_datetime( df['date'] ).dt.strftime( '%Y-%U' )


In [41]:
# ----------------------------
# Filtering
# ----------------------------
# Widget to control data
date_limit = widgets.SelectionSlider(
                options = df['date'].sort_values().unique().tolist(),
                value='2014-12-01',
                description='Max Available Date',
                disable=False,
                continuous_update=False,
                orientation='horizontal',
                style={'description_width': 'initial'},
                readout=True)

# Max Year Renovated
year_limit = widgets.SelectionSlider(
                options = df['yr_renovated'].sort_values().unique().tolist(),
                value=2000,
                description='Max Year',
                disable=False,
                continuous_update=False,
                orientation='horizontal',
                style={'description_width': 'initial'},
                readout=True
                )

# Waterfront
waterfront_limit = widgets.Checkbox(
                value=False,
                description='Waterfront?',
                disabled=False,
                indent=False
                )

In [49]:
def update_map( df, date_limit, year_limit, waterfront_limit ):
    # Filter data
    dtf = df[(df['date'] <= date_limit) &
        (df['yr_renovated'] >= year_limit) &
        (df['waterfront'] == waterfront_limit)].copy()

    fig = plt.figure( figsize=(24, 12) )

    specs = gridspec.GridSpec( ncols=2, nrows=2, figure=fig )

    ax1 = fig.add_subplot( specs[0, :] ) # First Row
    ax2 = fig.add_subplot( specs[1, 0] ) # First Row First Column
    ax3 = fig.add_subplot( specs[1, 1] ) # Second Row First Column

    by_year = dtf[['id', 'year']].groupby( 'year' ).sum().reset_index()
    ax1.bar( by_year['year'], by_year['id'] )

    # Frist Graph
    by_day = dtf[['id', 'date']].groupby( 'date' ).mean().reset_index()
    ax2.plot( by_day['date'], by_day['id'] )
    ax2.set_title( "Title: Avg Price by Day" )
    
    dtf['year_week'] = pd.to_datetime( dtf['date'] ).dt.strftime( '%Y-%U' )
    by_week_of_year = dtf[['id', 'year_week']].groupby( 'year_week' ).mean().reset_index()
    ax3.bar( by_week_of_year['year_week'], by_week_of_year['id'] )
    ax3.set_title( "Title: Avg Price by Week of Year" )
    
    plt.xticks( rotation=60);


In [50]:
widgets.interactive(    update_map,
                        df = fixed( df ),
                        date_limit=date_limit,
                        year_limit=year_limit,
                        waterfront_limit=waterfront_limit )

interactive(children=(SelectionSlider(continuous_update=False, description='Max Available Date', index=212, op…